In [1]:
# Dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pymongo
import requests
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
# Set up of Chrome webdriver
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240


[WDM] - Driver [/Users/brykobe24/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver] found in cache


In [3]:
# Scraping preparation and store data in dictionary
get_mars_data = {}
url = 'https://mars.nasa.gov/news/'
response= requests.get(url)
soup = bs(response.text, 'html.parser')

# NASA Mars News:
Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [4]:
news_title = soup.find('div', class_= 'content_title').text
news_paragraph = soup.find('div', class_= 'rollover_description_inner').text 
print('Most Recent Nasa News Article...')
print(f'Title: {news_title}')
print(f'Substance: {news_paragraph}')

# Push values to mars dictionary
get_mars_data['recent_news'] = news_title
get_mars_data['recent_news_substance'] = news_paragraph

Most Recent Nasa News Article...
Title: 

NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities


Substance: 
Starting July 27, news activities will cover everything from mission engineering and science to returning samples from Mars to, of course, the launch itself.



# JPL Mars Space Images - Featured Image
Visit the url for JPL Featured Space Image here. Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.

In [5]:
# Url we will be scraping images from
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
base_url = 'https://www.jpl.nasa.gov'

response = requests.get(url)
soup = bs(response.text, 'html.parser')

splint_url = base_url + soup.find('a', class_="button fancybox")["data-fancybox-href"]

print(f"URL to Featured Nasa Image: {splint_url}")


URL to Featured Nasa Image: https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA07137_ip.jpg


# Mars Facts
Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.Use Pandas to convert the data to a HTML table string.

In [6]:
url = 'https://space-facts.com/mars/'
# Read table data from url
facts_table = pd.read_html(url)

# Convert to dataframe
mars_facts_df = facts_table[0]
mars_facts_df.columns = ['Type', 'Measurement']
mars_facts_df

,Type,Measurement
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [7]:
#create HTML table
html_table = mars_facts_df.to_html(border=3)
#Remove enter characters 
get_mars_data['mars_facts_html'] = html_table.replace('\n', '')
print(get_mars_data['mars_facts_html'])

<table border="3" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Type</th>      <th>Measurement</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomer

# Mars Hemispheres
Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.Use Pandas to convert the data to a HTML table string.

In [8]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
base_url = "https://astrogeology.usgs.gov"
# Obtain the webpage
response = requests.get(url)
soup = bs(response.text, 'html.parser')
# Grab all image urls and append to list
results = soup.find_all('a', class_="itemLink product-item")
full_res_img_url = []
for result in results:
    # Combine link and base url
    full_res_img_url.append(base_url + result['href'])
    
print(full_res_img_url)

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']


In [ ]:
#create a empty list for diction
hem_img_urls = []
base_url = 'https://astrogeology.usgs.gov'

for url in full_res_img_url:
    #retrieve the webpage from the new website
    response = requests.get(url)
    soup = bs(response.text, 'html.parser')
    #Retrieve url to full resolution image
    image_url = soup.find('div', class_="downloads").find('ul').find('li').find('a')['href']
    #Retrieve the subject
    title = soup.find('h2', class_="title").text
    #initial diction and put into list
    resolution_dict = { "title":title,"img_url": image_url }
    hem_img_urls.append(resolution_dict)
    print(title)
    print(image_url)
    print("----------------------------------------------------------------")
print(hem_img_urls)
marsData['hemisphere_image_urls'] = hemisphere_image_urls
#print all data from diction 
print(marsData)